In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2
from keras.models import load_model

In [2]:
labels = ['PNEUMONIA', 'NORMAL']
img_size = 150
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))     # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [4]:
train = get_data('C:/Users/prajw/Downloads/pneumonia/dataset/train')
test = get_data('C:/Users/prajw/Downloads/pneumonia/dataset/test')
val = get_data('C:/Users/prajw/Downloads/pneumonia/dataset/val')

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (5216, 2) + inhomogeneous part.

In [5]:
train_dir = 'C:/Users/prajw/Downloads/pneumonia/dataset/train'
test_dir = 'C:/Users/prajw/Downloads/pneumonia/dataset/test'
val_dir = 'C:/Users/prajw/Downloads/pneumonia/dataset/val'

In [6]:
input_shape = (150, 150, 3)  # Input shape of the images
batch_size = 32
epochs = 10

In [7]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [8]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary'
)

Found 5216 images belonging to 2 classes.


In [10]:
validation_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary'
)

Found 16 images belonging to 2 classes.


In [12]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [13]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/10
163/163 [==============================] - 73s 403ms/step - loss: 0.4407 - accuracy: 0.8020
Epoch 2/10
163/163 [==============================] - 64s 392ms/step - loss: 0.2640 - accuracy: 0.8892
Epoch 3/10
163/163 [==============================] - 65s 395ms/step - loss: 0.2055 - accuracy: 0.9149
Epoch 4/10
163/163 [==============================] - 65s 397ms/step - loss: 0.1904 - accuracy: 0.9231
Epoch 5/10
163/163 [==============================] - 65s 397ms/step - loss: 0.1843 - accuracy: 0.9283
Epoch 6/10
163/163 [==============================] - 65s 397ms/step - loss: 0.1763 - accuracy: 0.9312
Epoch 7/10
163/163 [==============================] - 65s 398ms/step - loss: 0.1690 - accuracy: 0.9363
Epoch 8/10
163/163 [==============================] - 65s 395ms/step - loss: 0.1626 - accuracy: 0.9387
Epoch 9/10
163/163 [==============================] - 65s 399ms/step - loss: 0.1469 - accuracy: 0.9431
Epoch 10/10
163/163 [==============================] - 65s 398ms/step - l

In [15]:
# Evaluate the model on test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary'
)


Found 624 images belonging to 2 classes.


In [16]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print('Test accuracy:', test_acc) 

19/19 [==============================] - 5s 242ms/step - loss: 0.3537 - accuracy: 0.8947
Test accuracy: 0.8947368264198303


In [17]:
model.save('pneumonia_detection.h5')